In [46]:
from openai import OpenAI
import pandas as pd
import numpy as np
from slugify import slugify
from sklearn.metrics.pairwise import cosine_similarity

from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /Users/nealcaren/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [47]:
index_fn = f'/Users/nealcaren/Library/CloudStorage/Dropbox/gpt-ocr/index.md'
with open(index_fn, 'r') as infile:
    index_md = infile.read()

paragraphs = []
paragraphs_text = index_md.splitlines()
paragraphs_text = [line for line in paragraphs_text if line.strip() != '']
paragraphs_text = [line for line in paragraphs_text if line[:2]!='[^']
paragraphs_text = [line for line in paragraphs_text if line[:2]!='# ']
paragraphs_text = [line for line in paragraphs_text if '[' not in line]
paragraphs_text = [line for line in paragraphs_text if '::' not in line]
paragraphs_text = [line for line in paragraphs_text if '<!' not in line]

paragraphs_text = [line for line in paragraphs_text if len(line.split())>1]
        
for idx, paragraph in enumerate(
            paragraphs_text, start=1
        ):  # start=1 to start numbering from 1
            paragraphs.append(
                {"text": paragraph, "paragraph_number": idx, }
            )

In [48]:
idf=pd.DataFrame(paragraphs)

In [49]:
def get_chapter_from_page(page_number):
    # Define the ranges of page numbers for each chapter
    chapters = {
        1: range(1, 18),        # Chapter I: Pages 1-17
        2: range(18, 39),       # Chapter II: Pages 18-38
        3: range(39, 61),       # Chapter III: Pages 39-60
        4: range(61, 81),       # Chapter IV: Pages 61-80
        5: range(81, 101),      # Chapter V: Pages 81-100
        6: range(101, 126),     # Chapter VI: Pages 101-125
        7: range(126, 147),     # Chapter VII: Pages 126-146
        8: range(147, 167),     # Chapter VIII: Pages 147-166
        9: range(167, 193)      # Chapter IX: Pages 167-192
        # Note: Bibliography and Index are not counted as chapters
    }

    # Convert the page number to an integer
    page_number = str(page_number).replace('-','–')
    page_number = int(page_number.split('–')[0])

    # Find the chapter that the page number falls into
    for chapter, pages in chapters.items():
        if page_number in pages:
            return chapter

    # If the page number is not in any chapter range, return None
    return None

# Test the function with the provided example
get_chapter_from_page("111-119")


6

In [50]:
def process_string(input_strings):
    process_text = []
    for input_string in input_strings.split('; '):
        # Extract the last part of the string that contains the page number or range
            page_number_or_range = input_string.split(', ')[-1].strip()
             # Convert the page number to an integer
            page_number = str(page_number_or_range).replace('-','–')
            page_number = int(page_number.split('–')[0])

            # Get the chapter number using the page number or range
            chapter_number = get_chapter_from_page(page_number)

            # Slugify the rest of the string
            #slugified_text = slugify(text_without_page_number)

            # Format the new Markdown link
            #markdown_link = f'[{page_number_or_range}](chapter-{chapter_number}.md#{slugified_text})'
            markdown_link = f'[{page_number_or_range}](chapter-0{chapter_number}.md#Page_{page_number})'

            # Replace the page number or range at the end of the input string with the new Markdown link
            process_text.append(input_string.replace(page_number_or_range,'') + markdown_link)

    return '; '.join(process_text).replace('   ',' ')+'  SP   '


In [51]:
process_string('Adams, Henry, leader of the exodus to Kansas, 135-136')

'Adams, Henry, leader of the exodus to Kansas, [135-136](chapter-07.md#Page_135)  SP   '

In [52]:
idf['md_text'] = idf['text'].apply(process_string)

In [53]:
for _, row in idf.iterrows():
    index_md = index_md.replace(row['text'], row['md_text'])


In [54]:
sb = '''

::: {.small-break}
::: 

'''

index_slices = index_md.split('::::: {.index}\n')
index_slices[1] = index_slices[1].replace('\n\n',sb)
index_md2 = '::::: {.index}\n'.join(index_slices)
index_md2 = index_md2.replace('  SP   ','\n')

In [55]:
print(index_md2)

# Index {.unnumbered}


::::: {.index}

Adams, Henry, leader of the exodus to Kansas, [135](chapter-07.md#Page_135)

Akron, friends of fugitives in, [30](chapter-02.md#Page_30)

Alton Telegraph, comment of, [118](chapter-06.md#Page_118)

Anderson, promoter of settling of Negroes in Jamaica, [79](chapter-04.md#Page_79)

Anti-slavery, leaders of the movement, became more helpful to the refugees, 34, [35](chapter-02.md#Page_35)

Anti-slavery sentiment, of two kinds, [3](chapter-01.md#Page_3)

American Federation of Labor, attitude of, toward Negro labor, [191](chapter-09.md#Page_191)

Appalachian highland, settlers of, aided fugitives, [31-34](chapter-02.md#Page_31); exodus of Negroes to, [146](chapter-07.md#Page_146)

Arkansas, drain of laborers to, [120](chapter-06.md#Page_120)


::: {.small-break}
::: 

Ball, J. P., a contractor, [95](chapter-05.md#Page_95)

Ball, Thomas, a contractor, [95](chapter-05.md#Page_95)

Barclay, interest of, in the sending of Negroes to Jamaica, [79](chapter

In [56]:
with open('/Users/nealcaren/Documents/GitHub/a-century-of-negro-migration/chapters/chapter-11a-index.md' ,'w') as outfile:
    outfile.write(index_md2)